## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
## Importing relevant libraries

dbutils.library.installPyPI("mlflow")
dbutils.library.installPyPI("googletrans")
dbutils.library.installPyPI("textblob")
dbutils.library.restartPython()

In [3]:
import mlflow

from pyspark.sql.functions import isnan, when, count, col

from pyspark.sql import functions as F 
from pyspark.sql import Window
from pyspark.sql.functions import udf

from pyspark.sql.functions import countDistinct

import pandas as pd
import math

from pyspark.mllib.stat import Statistics

from googletrans import Translator
from textblob import TextBlob

from pyspark.ml.feature import CountVectorizer, StringIndexer, RegexTokenizer,StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Word2Vec

from pyspark.sql.functions import dayofweek
from pyspark.sql.functions import date_format
from pyspark.sql.functions import hour

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# To remove
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
## Importing train dataset

# File location and type
file_location = "/FileStore/tables/RS_v2_2006_03"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_train = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_train)

archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,brand_safe,can_gild,contest_mode,created_utc,distinguished,domain,edited,gilded,hidden,hide_score,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,post_hint,preview,retrieved_on,rte_mode,score,secure_media,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,url,whitelist_status
true,codepoet,null,null,null,List(),null,null,text,true,true,false,1141171234,null,macgeekery.com,false,0,false,false,2icw,true,false,false,false,null,List(),null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2icw/well_that_was_a_bust/,null,null,null,markdown,0,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Well That Was a Bust,http://www.macgeekery.com/opinion/well_that_was_a_bust,all_ads
true,scylla,null,null,null,List(),null,null,text,true,true,false,1141171723,null,msnbc.msn.com,false,0,false,false,2idn,true,false,false,false,null,List(),null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2idn/holocaust_why_david_irving_shouldnt_be_jailed_and/,null,null,null,markdown,0,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Holocaust: Why David Irving shouldn’t be jailed ( and it's not because he's in any ways right ),http://www.msnbc.msn.com/id/11569497/site/newsweek/,all_ads
true,tilto,null,null,null,List(),null,null,text,true,true,false,1141171939,null,iht.com,false,0,false,false,2ie4,true,false,false,false,null,List(),null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2ie4/google_shares_fall_sharply_as_cfo_announces/,null,null,null,markdown,0,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Google shares fall sharply as CFO announces growth is slowing,http://www.iht.com/articles/2006/02/28/business/google.php,all_ads
true,Laibcoms,null,null,null,List(),null,null,text,true,true,false,1141172196,null,gameshogun.info,false,0,false,false,2iek,true,false,false,false,null,List(),null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2iek/newsvine_launching_tomorrow/,null,null,null,markdown,0,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,newsvine launching tomorrow!,http://gameshogun.info/index.php/Tech/2006/03/01/newsvine_launching_tomorrow,all_ads
true,FaeLLe,null,null,null,List(),null,null,text,true,true,false,1141172277,null,faelle.com,false,0,false,false,2ies,true,false,false,false,null,List(),null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2ies/voodoopc_to_launch_8tb_media_pc/,null,null,null,markdown,0,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,VoodooPC to launch 8TB Media PC,http://www.FaeLLe.com/2006/03/voodoopc-plans-8tb-media-pc.html,all_ads
true,Megasphaera,null,null,null,List(),null,null,text,true,true,false,1141172696,null,request.reddit.com,false,0,false,false,2if8,true,false,false,false,null,List(),null,dark,text,false,null,true,1,0,false,all_ads,/r/reddit.com/comments/2if8/kafka_immigration/,null,null,null,markdown,0,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,kafka immigration,http://request.reddit.com/goto?id=2i9k,all_ads
true,alsaad,null,null,null,List(),null,null,text,false,true,false,1141173165,null,pandora.com,false,0,false,false,2ig1,false,false,false,false,null,List(),null,dark,text,false,null,true,0,0,false,null,/r/pl/comments/2ig1/pandoracom_wybieraj_oceniaj_i_sluchaj_tego_co/,null,null,null,markdown,4,null,,true,false,false,pl,t5_2475,r/pl,r

In [5]:
len(df_train.columns)

Out[3]: 58

In [6]:
'''Column that has a single value or total null values'''

drop_list1 = ['archived', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
               'contest_mode', 'distinguished', 'edited', 'gilded', 'hidden', 'hide_score', 'is_reddit_media_domain', 'is_self', 'is_video',
               'link_flair_css_class', 'link_flair_richtext', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media',
               'media_embed', 'num_crossposts', 'rte_mode', 'secure_media', 'secure_media_embed', 'selftext', 'send_replies', 'spoiler', 'stickied',
               ]

'''Columns that are too sparse (very less entries)'''

drop_list2 = ['thumbnail', 'thumbnail_height', 'thumbnail_width', 'post_hint', 'preview', 'author_cakeday', 'retrieved_on', 'author_flair_text_color',
             'suggested_sort', 'author_flair_type'] 

'''Columns that are redundant or correlated with other columns'''

drop_list3 = ['parent_whitelist_status', 'subreddit_id', 'subreddit_name_prefixed', 'permalink', 'id']

df_train = df_train.drop(*drop_list1)
df_train = df_train.drop(*drop_list2)
df_train = df_train.drop(*drop_list3)

In [7]:
display(df_train)

author,brand_safe,can_gild,created_utc,domain,is_crosspostable,no_follow,num_comments,over_18,score,subreddit,subreddit_type,title,url,whitelist_status
codepoet,true,true,1141171234,macgeekery.com,true,true,0,false,0,reddit.com,archived,Well That Was a Bust,http://www.macgeekery.com/opinion/well_that_was_a_bust,all_ads
scylla,true,true,1141171723,msnbc.msn.com,true,true,0,false,0,reddit.com,archived,Holocaust: Why David Irving shouldn’t be jailed ( and it's not because he's in any ways right ),http://www.msnbc.msn.com/id/11569497/site/newsweek/,all_ads
tilto,true,true,1141171939,iht.com,true,true,0,false,0,reddit.com,archived,Google shares fall sharply as CFO announces growth is slowing,http://www.iht.com/articles/2006/02/28/business/google.php,all_ads
Laibcoms,true,true,1141172196,gameshogun.info,true,true,0,false,0,reddit.com,archived,newsvine launching tomorrow!,http://gameshogun.info/index.php/Tech/2006/03/01/newsvine_launching_tomorrow,all_ads
FaeLLe,true,true,1141172277,faelle.com,true,true,0,false,0,reddit.com,archived,VoodooPC to launch 8TB Media PC,http://www.FaeLLe.com/2006/03/voodoopc-plans-8tb-media-pc.html,all_ads
Megasphaera,true,true,1141172696,request.reddit.com,true,true,1,false,0,reddit.com,archived,kafka immigration,http://request.reddit.com/goto?id=2i9k,all_ads
alsaad,false,true,1141173165,pandora.com,false,true,0,false,4,pl,restricted,"Pandora.com - wybieraj, oceniaj i sluchaj tego co lubisz. Za free.",http://pandora.com/,null
Megasphaera,true,true,1141173275,rxpgnews.com,true,true,0,false,0,reddit.com,archived,meditation changes brain structure,http://www.rxpgnews.com/research/neurosciences/article_2837.shtml,all_ads
benm,true,true,1141173366,blogs.pragprog.com,true,true,0,false,7,programming,public,Annotate Models Plugin for Rails,http://blogs.pragprog.com/cgi-bin/pragdave.cgi/Tech/Ruby/AnnotateModels.rdoc,all_ads
johnny_yuma,true,true,1141173368,cbsnews.com,true,false,1,false,9,reddit.com,archived,Kids Build Soybean-Fueled Car,http://www.cbsnews.com/stories/2006/02/17/eveningnews/main1329941.shtml,all_ads


In [8]:
len(df_train.columns)  # Now checking how many columns remain

Out[6]: 15

In [9]:
display(df_train.describe('score'))  # Getting some info on target column

summary,score
count,12525
mean,7.5120159680638725
stddev,29.10394679535981
min,0
max,583


In [10]:
df_train.printSchema()  # Getting info on schema of train dataset

root
-- author: string (nullable = true)
-- brand_safe: boolean (nullable = true)
-- can_gild: boolean (nullable = true)
-- created_utc: long (nullable = true)
-- domain: string (nullable = true)
-- is_crosspostable: boolean (nullable = true)
-- no_follow: boolean (nullable = true)
-- num_comments: long (nullable = true)
-- over_18: boolean (nullable = true)
-- score: long (nullable = true)
-- subreddit: string (nullable = true)
-- subreddit_type: string (nullable = true)
-- title: string (nullable = true)
-- url: string (nullable = true)
-- whitelist_status: string (nullable = true)

In [11]:
## Importing test dataset

# File location and type
file_location = "/FileStore/tables/RS_v2_2006_04-1"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_test = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_test)

archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,brand_safe,can_gild,contest_mode,created_utc,distinguished,domain,edited,gilded,hidden,hide_score,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,post_hint,preview,retrieved_on,rte_mode,score,secure_media,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,url,whitelist_status
true,spif,null,null,null,List(),null,null,text,true,true,false,1143849794,null,democrats.reform.house.gov,false,0,false,false,3rip,true,false,false,false,null,List(),null,dark,text,false,null,false,0,0,false,all_ads,/r/reddit.com/comments/3rip/iraq_on_the_record_a_searchable_collection_of_237/,null,null,null,markdown,4,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Iraq on the Record: A Searchable Collection Of 237 Specific Misleading Statements Made By Bush Administration Officials About The Threat Posed By Iraq,http://democrats.reform.house.gov/IraqOnTheRecord/,all_ads
true,jjzak,null,null,null,List(),null,null,text,false,true,false,1143849822,null,rubricks.org,false,0,false,false,3rir,true,false,false,false,null,List(),null,dark,text,false,null,false,0,0,false,null,/r/ja/comments/3rir/rubricks_cms_on_rails/,null,null,null,markdown,3,null,,true,false,false,ja,t5_22i6,r/ja,public,new,default,null,null,Rubricks - CMS on Rails,http://rubricks.org/,null
true,toddieg,null,null,null,List(),null,null,text,true,true,false,1143849849,null,randomcraponline.com,false,0,false,false,3ris,false,false,false,false,null,List(),null,dark,text,false,null,true,1,0,false,all_ads,/r/reddit.com/comments/3ris/islamic_dance_party/,null,null,null,markdown,0,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Islamic Dance Party,http://www.randomcraponline.com/islamic-dance-party/,all_ads
true,rmc,null,null,null,List(),null,null,text,true,true,false,1143849910,null,debian-administration.org,false,0,false,false,3riv,true,false,false,false,null,List(),null,dark,text,false,null,false,1,0,false,all_ads,/r/reddit.com/comments/3riv/how_to_make_bash_complete_differently_based_on/,null,null,null,markdown,3,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,How to make bash complete differently based on the command.,http://www.debian-administration.org/articles/317,all_ads
true,toddieg,null,null,null,List(),null,null,text,false,true,false,1143850224,null,randomcraponline.com,false,0,false,false,3rj0,false,false,false,false,null,List(),null,dark,text,false,null,true,0,0,true,promo_adult_nsfw,/r/nsfw/comments/3rj0/hot_web_cam_chicks/,null,null,null,markdown,0,null,,true,false,false,nsfw,t5_vf2,r/nsfw,public,null,default,null,null,Hot Web Cam Chicks,http://www.randomcraponline.com/web-cam-chicks/?bikini-thong/,promo_adult_nsfw
true,tiagocardoso,null,null,null,List(),null,null,text,true,true,false,1143850300,null,mainada.net,false,0,false,false,3rj3,true,false,false,false,null,List(),null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/3rj3/manga_style_other_sketches_see_the_artist_creation/,null,null,null,markdown,0,null,,true,false,false,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Manga Style & other sketches - see the artist creation,http://www.mainada.net/comics/index/mangastyle,all_ads
true,neotrantor,null,null,null,List(),null,null,text,true,true,false,1143850524,null,blog.cognitivelabs.com,false,0,false,false,3rj5,true,false,false,false,null,List(),null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/3rj5/scent_of_fear_improves_cognitive_performance_in/,null,nu

In [12]:
len(df_test.columns)

Out[10]: 58

In [13]:
'''Column that has a single value or total null values'''

drop_list1 = ['archived', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
               'contest_mode', 'distinguished', 'edited', 'gilded', 'hidden', 'hide_score', 'is_reddit_media_domain', 'is_self', 'is_video',
               'link_flair_css_class', 'link_flair_richtext', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media',
               'media_embed', 'num_crossposts', 'rte_mode', 'secure_media', 'secure_media_embed', 'selftext', 'send_replies', 'spoiler', 'stickied',
               ]

'''Columns that are too sparse (very less entries)'''

drop_list2 = ['thumbnail', 'thumbnail_height', 'thumbnail_width', 'post_hint', 'preview', 'author_cakeday', 'retrieved_on', 'author_flair_text_color',
             'suggested_sort', 'author_flair_type'] 

'''Columns that are redundant or correlated with other columns'''

drop_list3 = ['parent_whitelist_status', 'subreddit_id', 'subreddit_name_prefixed', 'permalink', 'id']

df_test = df_test.drop(*drop_list1)
df_test = df_test.drop(*drop_list2)
df_test = df_test.drop(*drop_list3)

In [14]:
display(df_test)

author,brand_safe,can_gild,created_utc,domain,is_crosspostable,no_follow,num_comments,over_18,score,subreddit,subreddit_type,title,url,whitelist_status
spif,true,true,1143849794,democrats.reform.house.gov,true,false,0,false,4,reddit.com,archived,Iraq on the Record: A Searchable Collection Of 237 Specific Misleading Statements Made By Bush Administration Officials About The Threat Posed By Iraq,http://democrats.reform.house.gov/IraqOnTheRecord/,all_ads
jjzak,false,true,1143849822,rubricks.org,true,false,0,false,3,ja,public,Rubricks - CMS on Rails,http://rubricks.org/,null
toddieg,true,true,1143849849,randomcraponline.com,false,true,1,false,0,reddit.com,archived,Islamic Dance Party,http://www.randomcraponline.com/islamic-dance-party/,all_ads
rmc,true,true,1143849910,debian-administration.org,true,false,1,false,3,reddit.com,archived,How to make bash complete differently based on the command.,http://www.debian-administration.org/articles/317,all_ads
toddieg,false,true,1143850224,randomcraponline.com,false,true,0,true,0,nsfw,public,Hot Web Cam Chicks,http://www.randomcraponline.com/web-cam-chicks/?bikini-thong/,promo_adult_nsfw
tiagocardoso,true,true,1143850300,mainada.net,true,true,0,false,0,reddit.com,archived,Manga Style & other sketches - see the artist creation,http://www.mainada.net/comics/index/mangastyle,all_ads
neotrantor,true,true,1143850524,blog.cognitivelabs.com,true,true,0,false,0,reddit.com,archived,Scent of Fear Improves Cognitive Performance in Women,http://blog.cognitivelabs.com/2006/03/scent-of-fear-improves-cognitive.html,all_ads
honekaimedia,true,true,1143850569,techeblog.com,true,true,1,false,0,reddit.com,archived,Top 10 Strangest Cell Phones,http://www.techeblog.com/index.php/tech-gadget/top-10-strangest-cell-phones,all_ads
eglobe1,true,true,1143850981,eglobe1.com,false,true,0,false,0,reddit.com,archived,ten crazy usb,http://www.eglobe1.com/index.php/2006/03/31/10-crazy-usb/,all_ads
otsune,false,true,1143851173,bsddiary.net,true,true,0,false,3,ja,public,bind9 設定,http://www.bsddiary.net/doc/bind9.html,null


In [15]:
len(df_test.columns)  # Checking to see how many columns remain in testing datset

Out[13]: 15

In [16]:
# Imputing blank values as "no_status: in whitelist_status column in both train and test dataset

df_train = df_train.fillna( {'whitelist_status':'no_status'} )
df_test = df_test.fillna( {'whitelist_status':'no_status'} )

In [17]:
# Checking to see if any null values remain in train dataset

display(df_train.select([count(when(col(c).isNull(), c)).alias(c) for c in df_train.columns]))

author,brand_safe,can_gild,created_utc,domain,is_crosspostable,no_follow,num_comments,over_18,score,subreddit,subreddit_type,title,url,whitelist_status
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
# Checking to see if any null values remain in test dataset

display(df_test.select([count(when(col(c).isNull(), c)).alias(c) for c in df_test.columns]))

author,brand_safe,can_gild,created_utc,domain,is_crosspostable,no_follow,num_comments,over_18,score,subreddit,subreddit_type,title,url,whitelist_status
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
# Checking distinct values in each column of training set

display(df_train.agg(*(countDistinct(col(c)).alias(c) for c in df_train.columns)))

author,brand_safe,can_gild,created_utc,domain,is_crosspostable,no_follow,num_comments,over_18,score,subreddit,subreddit_type,title,url,whitelist_status
2360,2,2,12496,5204,2,2,59,2,230,23,3,12383,12354,3


In [20]:
# Boxplot for score column

display(df_train)

author,brand_safe,can_gild,created_utc,domain,is_crosspostable,no_follow,num_comments,over_18,score,subreddit,subreddit_type,title,url,whitelist_status
codepoet,true,true,1141171234,macgeekery.com,true,true,0,false,0,reddit.com,archived,Well That Was a Bust,http://www.macgeekery.com/opinion/well_that_was_a_bust,all_ads
scylla,true,true,1141171723,msnbc.msn.com,true,true,0,false,0,reddit.com,archived,Holocaust: Why David Irving shouldn’t be jailed ( and it's not because he's in any ways right ),http://www.msnbc.msn.com/id/11569497/site/newsweek/,all_ads
tilto,true,true,1141171939,iht.com,true,true,0,false,0,reddit.com,archived,Google shares fall sharply as CFO announces growth is slowing,http://www.iht.com/articles/2006/02/28/business/google.php,all_ads
Laibcoms,true,true,1141172196,gameshogun.info,true,true,0,false,0,reddit.com,archived,newsvine launching tomorrow!,http://gameshogun.info/index.php/Tech/2006/03/01/newsvine_launching_tomorrow,all_ads
FaeLLe,true,true,1141172277,faelle.com,true,true,0,false,0,reddit.com,archived,VoodooPC to launch 8TB Media PC,http://www.FaeLLe.com/2006/03/voodoopc-plans-8tb-media-pc.html,all_ads
Megasphaera,true,true,1141172696,request.reddit.com,true,true,1,false,0,reddit.com,archived,kafka immigration,http://request.reddit.com/goto?id=2i9k,all_ads
alsaad,false,true,1141173165,pandora.com,false,true,0,false,4,pl,restricted,"Pandora.com - wybieraj, oceniaj i sluchaj tego co lubisz. Za free.",http://pandora.com/,no_status
Megasphaera,true,true,1141173275,rxpgnews.com,true,true,0,false,0,reddit.com,archived,meditation changes brain structure,http://www.rxpgnews.com/research/neurosciences/article_2837.shtml,all_ads
benm,true,true,1141173366,blogs.pragprog.com,true,true,0,false,7,programming,public,Annotate Models Plugin for Rails,http://blogs.pragprog.com/cgi-bin/pragdave.cgi/Tech/Ruby/AnnotateModels.rdoc,all_ads
johnny_yuma,true,true,1141173368,cbsnews.com,true,false,1,false,9,reddit.com,archived,Kids Build Soybean-Fueled Car,http://www.cbsnews.com/stories/2006/02/17/eveningnews/main1329941.shtml,all_ads


In [21]:
# Plotting histogram for score column

histogram_score = df_train.select('score').rdd.flatMap(lambda x: x).histogram(sc.parallelize(range(0, 601, 25)).collect())

# Loading the Computed Histogram into a Pandas Dataframe for plotting
pd.DataFrame(list(zip(*histogram_score)), columns=['bin', 'frequency']).set_index('bin').plot(kind='bar')

In [22]:
# Plotting correlation matrix

features = df_train.select(["brand_safe", "can_gild", "is_crosspostable", "no_follow", "num_comments", "over_18", "score"]).rdd.map(lambda row: row[0:])

corr_mat=Statistics.corr(features, method="pearson")

sc.parallelize(corr_mat).map(lambda x: x.tolist()).toDF(["brand_safe", "can_gild", "is_crosspostable", "no_follow", "num_comments", "over_18", "score"]).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 brand_safe| can_gild| is_crosspostable| no_follow| num_comments| over_18| score|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 1.0| -0.0701705662321684|0.006550678297479...|-0.12803144883207107| 0.06753569504788715|-0.42584568088470176| 0.09017058006151887|
 -0.0701705662321684| 1.0| 0.2581224560804196| 0.06460756477445813|-0.02466470463248...| 0.00473092126426736|-0.03205051002258151|
0.006550678297479...| 0.2581224560804196| 1.0|-0.14138707721783444|0.040117687841533696|-0.01748743175010616| 0.06920129610356036|
-0.12803144883207107| 0.06460756477445813|-0.14138707721783444| 1.0| -0.2542872628694801| 0.11188963408762627| -0.4100765403605492|
 0.06753569504788715|-0.02466470463248...|0.040117687841533696| -0.2542872628694801| 1.0|-0.02987621180460...| 0.614284261344888|
-0.42584568088470176| 0.00473092126426736|-0.01748743175010616| 0.11188963408762627|-0.02987621180460...| 1.0|-0.02660469000276451|
 0.09017058006151887|-0.03205051002258151| 0.06920129610356036| -0.4100765403605492| 0.614284261344888|-0.02660469000276451| 1.0|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [23]:
# Looking at the count of posts from each subreddit

display(df_train.groupby('subreddit').count())

subreddit,count
vi,21
pl,35
programming,641
reddit.com,8503
tr,83
de,666
es,12
nsfw,330
request,113
it,24


In [24]:
def translation(x):
  
  translator = Translator()
  return translator.translate(str(x), dest = 'en').text

trans = udf(translation)
spark.udf.register('trans', trans)

def entropy(string):
  "Calculates the Shannon entropy of a string"
  string = string.strip()
  # get probability of chars in string
  prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]

  # calculate the entropy
  entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])

  return entropy

entro = udf(entropy)
spark.udf.register('entro', entro)

def numDigits(string):
  digits = [i for i in string if i.isdigit()]
  return len(digits)

digit = udf(numDigits)
spark.udf.register('digit', digit)

Out[22]: <function __main__.numDigits(string)>

In [25]:
## Feature Engineering & Transformations in Train Dataset

# Translating Title Column
df_train_trans = df_train.withColumn('title_translated', trans('title'))

# Performing Sentiment Analysis on Translated Title
sent = udf(lambda x: TextBlob(x).sentiment[0])
spark.udf.register('sentiment', sent)
df_train_trans = df_train_trans.withColumn('title_sentiment',sent('title_translated').cast('double'))

# Computing average score over each author and domain
df_train_trans = df_train_trans.withColumn('avg_author_score', F.avg('score').over(Window.partitionBy('author')))
df_train_trans = df_train_trans.withColumn('domain_avg_score', F.avg('score').over(Window.partitionBy('domain')))

# Tokenizing translated title
regex_tokenizer = RegexTokenizer(inputCol="title_translated", outputCol="title_tokenized", pattern="\\W")
df_train_trans = regex_tokenizer.transform(df_train_trans)

# Removing stopwords from tokenized title
remover = StopWordsRemover(inputCol="title_tokenized", outputCol="title_stopwords_removed")
df_train_trans = remover.transform(df_train_trans)

# Generating word-count, length, avg word length features on title
df_train_trans = df_train_trans.withColumn('wordCount', F.size(F.split(F.col('title_translated'), ' ')))
df_train_trans = df_train_trans.withColumn("length_of_title", F.length("title_translated"))
df_train_trans = df_train_trans.withColumn('avg_word_length', (F.col('length_of_title')-F.col('wordCount')+1)/F.col('wordCount'))

# TF-IDF on Title (stopwords removed)
hashingTF = HashingTF(inputCol="title_stopwords_removed", outputCol="title_tf", numFeatures=15)
featurizedData = hashingTF.transform(df_train_trans)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="title_tf", outputCol="title_tf_idf")
idfModel = idf.fit(featurizedData)
df_train_trans = idfModel.transform(featurizedData)

# Word2Vec generation on Title (stopwords removed)
word2Vec = Word2Vec(vectorSize=5, minCount=1, inputCol="title_stopwords_removed", outputCol="title_word2vec")
model_w2v = word2Vec.fit(df_train_trans)
df_train_trans = model_w2v.transform(df_train_trans)

# Feature Engineering on "created_utc" column
df_train_trans = df_train_trans.withColumn("date",F.to_timestamp(df_train_trans["created_utc"]))
df_train_trans = df_train_trans.withColumn("day_of_week", date_format(col('date'), 'EEEE'))
df_train_trans = df_train_trans.withColumn("hour", hour(col('date')))
df_train_trans = df_train_trans.withColumn("week_day", dayofweek(df_train_trans.date))

# Feature Engineering on "url" column
df_train_trans = df_train_trans.withColumn('entropy', entro('url').cast('float'))
df_train_trans = df_train_trans.withColumn('no_of_digits', digit('url').cast('float'))
df_train_trans = df_train_trans.withColumn('url_length', F.length('url').cast('int'))

In [26]:
display(df_train_trans.groupby('hour').avg('score'))

hour,avg(score)
12,9.298097251585624
22,8.07569721115538
1,8.968992248062015
13,6.537151702786378
6,6.889502762430939
16,8.722677595628415
3,6.482093663911845
20,8.952380952380953
5,5.9059304703476485
19,9.530551415797317


In [27]:
df_train_trans.groupby('day_of_week').avg('score').show()

+-----------+-----------------+
day_of_week| avg(score)|
+-----------+-----------------+
 Wednesday|7.658127619934793|
 Tuesday| 7.59800111049417|
 Friday|7.214076246334311|
 Thursday| 6.96597510373444|
 Saturday|7.149162861491629|
 Monday|7.168475157053113|
 Sunday|9.912878787878787|
+-----------+-----------------+

In [28]:
## Feature Engineering & Transformations in Test Dataset

# Translating Title Column
df_test_trans = df_test.withColumn('title_translated', trans('title'))

# Performing Sentiment Analysis on Translated Title
df_test_trans = df_test_trans.withColumn('title_sentiment',sent('title_translated').cast('double'))

# Tokenizing translated title
df_test_trans = regex_tokenizer.transform(df_test_trans)

# Removing stopwords from tokenized title
df_test_trans = remover.transform(df_test_trans)

# Computing average score over each author and domain
df_test_trans = df_test_trans.withColumn('avg_author_score', F.avg('score').over(Window.partitionBy('author')))
df_test_trans = df_test_trans.withColumn('domain_avg_score', F.avg('score').over(Window.partitionBy('domain')))

# Generating word-count, length, avg word length features on Title
df_test_trans = df_test_trans.withColumn('wordCount', F.size(F.split(F.col('title_translated'), ' ')))
df_test_trans = df_test_trans.withColumn("length_of_title", F.length("title_translated"))
df_test_trans = df_test_trans.withColumn('avg_word_length', (F.col('length_of_title')-F.col('wordCount')+1)/F.col('wordCount'))

# TF-IDF on Title (stopwords removed)
hashingTF = HashingTF(inputCol="title_stopwords_removed", outputCol="title_tf", numFeatures=15)
featurizedData = hashingTF.transform(df_test_trans)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="title_tf", outputCol="title_tf_idf")
idfModel = idf.fit(featurizedData)
df_test_trans = idfModel.transform(featurizedData)

# Word2Vec generation on Title (stopwords removed)
word2Vec = Word2Vec(vectorSize=5, minCount=1, inputCol="title_stopwords_removed", outputCol="title_word2vec")
model_w2v = word2Vec.fit(df_test_trans)
df_test_trans = model_w2v.transform(df_test_trans)

# Feature Engineering on "created_utc" column
df_test_trans = df_test_trans.withColumn("date",F.to_timestamp(df_test_trans["created_utc"]))
df_test_trans = df_test_trans.withColumn("day_of_week", date_format(col('date'), 'EEEE'))
df_test_trans = df_test_trans.withColumn("hour", hour(col('date')))
df_test_trans =  df_test_trans.withColumn("week_day", dayofweek(df_test_trans.date))

# Feature Engineering on "url" column
df_test_trans = df_test_trans.withColumn('entropy', entro('url').cast('float'))
df_test_trans = df_test_trans.withColumn('no_of_digits', digit('url').cast('float'))
df_test_trans = df_test_trans.withColumn('url_length', F.length('url').cast('int'))

In [29]:
# https://medium.com/analytics-vidhya/congressional-tweets-using-sentiment-analysis-to-cluster-members-of-congress-in-pyspark-10afa4d1556e

In [30]:
# Checking Correlation amongst generated and original features

df_train_trans_pd = df_train_trans.toPandas()
plt.figure(figsize = (15,15))
sns.heatmap(df_train_trans_pd.corr(), annot = True, fmt = '.2g')

In [31]:
# Saving the dataframe with all original and generated features (in parquet format)

df_test_trans.select("*").write.save("df_test_trans_aug2.parquet", format = 'parquet')

In [32]:
df_train_trans.select("*").write.save("df_train_trans_aug2.parquet", format = 'parquet')

In [33]:
# Loading the saved train and test (parquet) files

df_train_trans = spark.read.parquet("df_train_trans_aug2.parquet")
df_test_trans = spark.read.parquet("df_test_trans_aug2.parquet")

In [34]:
# Initializing assembler to tie-up all relevant features together

# vectorAssembler = VectorAssembler(inputCols = ['brand_safe', 'can_gild', 'is_crosspostable', 'no_follow', "avg_author_score", "domain_avg_score", \
#                                                "num_comments", 'over_18','wordCount', 'length_of_title', 'avg_word_length', 'title_tf_idf', \
#                                                'title_word2vec', 'week_day', 'entropy', 'no_of_digits', 'url_length' ]\
#                                   , outputCol = 'features')
vectorAssembler = VectorAssembler(inputCols = ['no_follow', "num_comments", "avg_author_score", "domain_avg_score"], outputCol = 'features')

In [35]:
# Assembling all stated features above on train and test set

df_train_assem = vectorAssembler.transform(df_train_trans)
df_test_assem = vectorAssembler.transform(df_test_trans)

In [36]:
# Fitting Linear Regression

lr = LinearRegression(featuresCol = 'features', labelCol='score', maxIter=1000, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(df_train_assem)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

trainingSummary = lr_model.summary
# On Train Set
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

lr_predictions = lr_model.transform(df_test_assem)
lr_predictions.select("prediction","score","features").show(5)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="score",metricName="r2")

# On Test Set
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

test_result = lr_model.evaluate(df_test_assem)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Coefficients: [-7.638988342542023,1.9158774928116922,0.36417668962299843,0.5895233158603375]
Intercept: 4.4019488651177054
RMSE: 17.522472
r2: 0.637489
+--------------------+-----+--------------------+
 prediction|score| features|
+--------------------+-----+--------------------+
 -1.2671746562817345| 0|[1.0,0.0,5.409090...|
-0.36746197912928924| 2| [1.0,1.0,1.0,1.0]|
 -2.283339471940981| 0| [1.0,0.0,1.0,1.0]|
 -2.751470557926986| 0|[1.0,0.0,1.333333...|
 -1.3211619846126252| 0| [1.0,1.0,0.0,0.0]|
+--------------------+-----+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.656071
Root Mean Squared Error (RMSE) on test data = 17.207

In [37]:
# Fitting RandomForestRegressor

rf = RandomForestRegressor(featuresCol='features') # featuresCol="indexedFeatures",numTrees=2, maxDepth=2, seed=42

df_train_sel = df_train_assem.select(col("features"), col("score").alias("label"))
df_train_sel.show(3)

df_test_sel = df_test_assem.select(col("features"), col("score").alias("label"))
df_test_sel.show(3)

model = rf.fit(df_train_sel)

predictions = model.transform(df_test_sel)

# Select example rows to display.
predictions.select("features","label", "prediction").show(5)

rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(predictions))

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+--------------------+-----+
 features|label|
+--------------------+-----+
[1.0,0.0,1.672597...| 0|
[0.0,0.0,2.020408...| 3|
 [1.0,3.0,0.0,0.0]| 0|
+--------------------+-----+
only showing top 3 rows

+--------------------+-----+
 features|label|
+--------------------+-----+
[1.0,0.0,5.409090...| 0|
 [1.0,1.0,1.0,1.0]| 2|
 [1.0,0.0,1.0,1.0]| 0|
+--------------------+-----+
only showing top 3 rows

+--------------------+-----+-------------------+
 features|label| prediction|
+--------------------+-----+-------------------+
[1.0,0.0,5.409090...| 0| 0.704431002916681|
 [1.0,1.0,1.0,1.0]| 2| 0.4513787113069898|
 [1.0,0.0,1.0,1.0]| 0| 0.4268540929580381|
[1.0,0.0,1.333333...| 0| 0.3687147535143388|
 [1.0,1.0,0.0,0.0]| 0|0.23586989070331424|
+--------------------+-----+-------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.681585
Root Mean Squared Error (RMSE) on test data = 16.5565

In [38]:
# https://runawayhorse001.github.io/LearningApacheSpark/regression.html#random-forest-regression

In [39]:
# Fitting DecisionTreeRegressor Model

dt = DecisionTreeRegressor(featuresCol="features")

model_dt = dt.fit(df_train_sel)

predictions_dt = model_dt.transform(df_test_sel)

predictions_dt.select("features","label","prediction").show(5)

print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(predictions_dt))

rmse_dt = evaluator.evaluate(predictions_dt)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_dt)

+--------------------+-----+-------------------+
 features|label| prediction|
+--------------------+-----+-------------------+
[1.0,0.0,5.409090...| 0|0.02326091727013386|
 [1.0,1.0,1.0,1.0]| 2| 0.6316377864728899|
 [1.0,0.0,1.0,1.0]| 0| 0.6316377864728899|
[1.0,0.0,1.333333...| 0|0.02326091727013386|
 [1.0,1.0,0.0,0.0]| 0|0.02326091727013386|
+--------------------+-----+-------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.630435
Root Mean Squared Error (RMSE) on test data = 17.8368

In [40]:
# from pyspark.ml.regression import GBTRegressor

# # Define LinearRegression algorithm
# gbt = GBTRegressor() #numTrees=2, maxDepth=2, seed=42

# model_gbt = gbt.fit(df_tt)

# predictions_gbt = model_gbt.transform(df_tt2)

# predictions_gbt.select("features","label","prediction").show(5)

# print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(predictions_gbt))

# rmse_gbt = evaluator.evaluate(predictions_gbt)
# print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_gbt)